# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**


#### **Nombres y matrículas de los integrantes del equipo:**
#### Equipo 88
*   Carlos Emilio Romero Polanco  - A01632239
*   Saavedra Solis Osiris Xcaret - A01795992
*   Amiel Joel Rosete Islas - A01748598
*   Marco Antonio Romero Brito - A01795934


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

In [ ]:
!pip install transformers accelerate
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import librosa   # paquete para audio y música
import json

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')

import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
# para filtrar advertencias:
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-01.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-04.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-05.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-06.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-14.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-22.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-24.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-25.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-26.mp3"
!gdown "https://www.gutenberg.org/files/21144/mp3/21144-27.mp3"



Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-01.mp3
To: /content/21144-01.mp3
100% 1.16M/1.16M [00:01<00:00, 859kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-04.mp3
To: /content/21144-04.mp3
100% 1.08M/1.08M [00:01<00:00, 786kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-05.mp3
To: /content/21144-05.mp3
100% 936k/936k [00:01<00:00, 698kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-06.mp3
To: /content/21144-06.mp3
100% 1.15M/1.15M [00:01<00:00, 848kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-14.mp3
To: /content/21144-14.mp3
100% 932k/932k [00:01<00:00, 811kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-22.mp3
To: /content/21144-22.mp3
100% 918k/918k [00:01<00:00, 681kB/s]
Downloading...
From: https://www.gutenberg.org/files/21144/mp3/21144-24.mp3
To: /content/21144-24.mp3
100% 1.03M/1.03M [00:01<00:00, 752kB/s]
Downloading.

In [ ]:
audio01, sample_rate = librosa.load("./21144-01.mp3")
audio04, sample_rate = librosa.load("./21144-04.mp3")
audio05, sample_rate = librosa.load("./21144-05.mp3")
audio06, sample_rate = librosa.load("./21144-06.mp3")
audio14, sample_rate = librosa.load("./21144-14.mp3")
audio22, sample_rate = librosa.load("./21144-22.mp3")
audio24, sample_rate = librosa.load("./21144-24.mp3")
audio25, sample_rate = librosa.load("./21144-25.mp3")
audio26, sample_rate = librosa.load("./21144-26.mp3")
audio27, sample_rate = librosa.load("./21144-27.mp3")

audios = [audio01, audio04, audio05, audio06, audio14, audio22, audio24, audio25, audio26, audio27]

# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

#model_id ="openai/whisper-tiny"    # el más sencillo y ligero, pero menos preciso.
#model_id ="openai/whisper-small"
#model_id ="openai/whisper-base"
#model_id ="openai/whisper-medium"  # Generalmente el modelo intermedio en cuanto a desempeño y tamaño.
#model_id = "openai/whisper-large-v3"    # mejor modelo
model_id = "openai/whisper-large-v3-turbo"    # una versión ligera de v3

# Cargamos el modelo:
processor = AutoProcessor.from_pretrained(model_id)
modelo = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

In [ ]:
# Inicializamos el pipeline:
pipe = pipeline("automatic-speech-recognition",
                model=modelo,
                 tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                torch_dtype=torch.float16,      #torch.float16 if torch.cuda.is_available() else torch.float32,
                device=device
                )

Device set to use cuda


In [ ]:
transcripciones = []


In [ ]:
for audio_path in audios:
  text = pipe(audio_path,
                return_timestamps=True,   # Para audios mayores a 30 segs.
                generate_kwargs={"language":"es"}  # Lo detecta automático, pero se le puede indicar el idioma.
                )
  transcripciones.append(text["text"])

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [ ]:
transcripciones

[' Las Fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo Un pequeño corderito decidió refugiarse en un templo cercano Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro Lo inmolaría a su dios Mejor así, replicó el cordero Prefiero ser víctima para un dios A tener que perecer en tus colmillos Si sin remedio vamos a ser sacrificados Más nos vale que sea con el mayor honor Fin de la fábula Esta es una grabación del dominio público',
 ' Las Fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz Fábula Número 64 El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de 

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
with open('transcripciones.json', 'w') as f:
    json.dump(transcripciones, f)

print("JSON creado")

JSON creado


In [ ]:
transcripciones1 = []
with open('transcripciones.json', 'r') as f:
    transcripciones1 = json.load(f)

In [ ]:
for i in range(len(transcripciones1)):
    index = transcripciones1[i].find('Fin')
    if index != -1:
        transcripciones1[i] = transcripciones1[i][:index].strip()  # without 'fin'
        # or keep 'fin' with: transcriptions[i][:index + len('fin')]

for i in range(len(transcripciones1)):
    words = transcripciones1[i].split()
    transcripciones1[i] = ' '.join(words[13:])

In [ ]:
transcripciones1

## Para 2 Fabulas, sigue existiendo el numero de audio que son, estas se limpiraran en el siguiente paso.

['El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo Un pequeño corderito decidió refugiarse en un templo cercano Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro Lo inmolaría a su dios Mejor así, replicó el cordero Prefiero ser víctima para un dios A tener que perecer en tus colmillos Si sin remedio vamos a ser sacrificados Más nos vale que sea con el mayor honor',
 '64 El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. Pidió entonces la cancelación de la paga convenida. Oye amiga, dijo el lobo, ¿no crees que es suficiente paga con haber sacado tu cabeza sana y salva de mi boca? Nunca hagas favores a malvados, traficantes o 

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

## Las stopwords a eliminar junto con la limpieza de caracteres y convertir a minusculas.
set1 = set(stopwords.words('spanish'))
mystopwords = list(set1)

def clean_tok(doc):

    # Tokenización y convertir todo a minúsculas
    tokens = re.findall(r'\b\w+\b', doc.lower())

    # Eliminar caracteres no alfabéticos y palabras de longitud mayor a 1
    tokens = [token for token in tokens if token.isalpha() and len(token) > 1]

    # Eliminar stopwords
    tokens = [token for token in tokens if token not in mystopwords]


    return tokens



In [ ]:
Xcleantok = [clean_tok(x) for x in transcripciones1]

In [ ]:
for x in Xcleantok[0:10]:
  print(x)

['lobo', 'cordero', 'templo', 'dándose', 'cuenta', 'perseguido', 'lobo', 'pequeño', 'corderito', 'decidió', 'refugiarse', 'templo', 'cercano', 'llamó', 'lobo', 'dijo', 'si', 'sacrificador', 'encontraba', 'allí', 'adentro', 'inmolaría', 'dios', 'mejor', 'así', 'replicó', 'cordero', 'prefiero', 'ser', 'víctima', 'dios', 'tener', 'perecer', 'colmillos', 'si', 'remedio', 'vamos', 'ser', 'sacrificados', 'vale', 'mayor', 'honor']
['lobo', 'coruja', 'lobo', 'comía', 'hueso', 'atragantó', 'hueso', 'garganta', 'corría', 'todas', 'partes', 'busca', 'auxilio', 'encontró', 'correr', 'grulla', 'pidió', 'salvara', 'aquella', 'situación', 'enseguida', 'pagaría', 'ello', 'aceptó', 'grulla', 'introdujo', 'cabeza', 'boca', 'lobo', 'sacando', 'garganta', 'hueso', 'atravesado', 'pidió', 'entonces', 'cancelación', 'paga', 'convenida', 'oye', 'amiga', 'dijo', 'lobo', 'crees', 'suficiente', 'paga', 'haber', 'sacado', 'cabeza', 'sana', 'salva', 'boca', 'nunca', 'hagas', 'favores', 'malvados', 'traficantes', '

# **Ejercicio 4:**

In [ ]:
spanish_stopwords = stopwords.words('spanish')
vectorizer = CountVectorizer(stop_words=spanish_stopwords)
X = vectorizer.fit_transform(transcripciones1)

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)



LatentDirichletAllocation(n_components=3, random_state=42)

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f" Tema {topic_idx}: ", end="")
        print(" + ".join([
            f"{feature_names[i]} ({topic[i]:.0f})" for i in topic.argsort()[:-num_top_words - 1:-1]
        ]))

num_top_words = 5
display_topics(lda, vectorizer.get_feature_names_out(), num_top_words)

 Tema 0: perro (8) + campanilla (4) + león (4) + dijo (3) + caballo (3)
 Tema 1: lobo (6) + reflejo (4) + perro (4) + si (3) + río (3)
 Tema 2: lobo (7) + paga (3) + hueso (3) + sino (2) + garganta (2)


# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
!pip install openai

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
from IPython.display import Markdown

from google.colab import userdata
from openai import OpenAI
import openai

In [ ]:
api_key = userdata.get("OpenAI_MNA")

if not api_key:
  raise ValueError("API key no encontrada en tus Secretos...")

client = OpenAI(api_key = api_key)

In [ ]:
system_prompt = """
Eres un lector de Fabulas donde vas a tener que leer y sacar conclusiones acerca del tema y resumir la Fabula.
"""
user_prompt_zs = "Dame 1 enunciado resumiendo la Fabula, luego 3 enunciados diferentes al primero relacionados con la Fabula."

def call_openai(system_prompt, user_prompt, model="gpt-4o-mini", temperature=0.7):
    response = client.chat.completions.create(model=model,
                                       temperature=temperature,
                                       max_tokens=1024,
                                       messages=[
                                           {"role": "system", "content": system_prompt},
                                           {"role": "user", "content": user_prompt}
                                           ]
    )
    return response.choices[0].message.content.strip()

In [ ]:
for i in range(len(Xcleantok)):
  print(f"Fabula {i+1}:")
  print("Iniciando el proceso Zero-Shot...\n")
  respuesta_zs = call_openai(system_prompt, user_prompt_zs)
  print(respuesta_zs)
  print("Resultado del Zero-Shot:\n")


Fabula 1:
Iniciando el proceso Zero-Shot...

**Resumen de la Fábula**: La fábula ilustra cómo la arrogancia y la falta de humildad pueden llevar a la ruina, mostrando la importancia de reconocer nuestras limitaciones y aprender de los demás.

**Enunciados relacionados**:
1. La historia nos enseña que la confianza excesiva en nuestras habilidades puede cegarnos ante los consejos valiosos de quienes nos rodean.
2. A través de los errores del protagonista, se destaca la necesidad de la modestia en la búsqueda del éxito.
3. La fábula resalta que la verdadera sabiduría proviene de la disposición a escuchar y aprender de las experiencias ajenas.
Resultado del Zero-Shot:

Fabula 2:
Iniciando el proceso Zero-Shot...

**Resumen de la Fábula:** La fábula enseña que la astucia y la inteligencia son más valiosas que la fuerza bruta.

**Enunciados relacionados:**
1. La historia muestra cómo un personaje ingenioso logra superar un obstáculo que parecía insuperable usando su mente en lugar de su pode

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**




| Modelo | Desempeño (lectura de texto) | Desempeño (generación resumen) | Ventajas | Desventajas | Conclusión |
|-----------|-----------|-----------|-----------|-----------|-----------|
| facebook/wav2vec2-large-xlsr-53-spanish | Regular, con algunos errores en la lectura de algunas palabras en español, con faltantes de algunas letras. <br>En la transcripción cambió algunas palabras al francés y le costo la generación de enunciados. En la salida<br> de los tokens estuvo bien y todas ellas hacían sentido | Al generar el resumen le costo un poco hacer correctamente el resumen y se podia entender el sentido del enunciado, <br>donde tuvo mas problemas es en la generación de los tres enunciados relacionados con la fábula y en ocasiones <br>cambiaba algunas palabras de idioma para darle sentido a la frase. | Es un modelo más sencillo y que se ejecuto de manera muy rápida, <br>util en transcripciones sencillas y no muy complejas. | Algunos errores en la transcripción a texto y le costo <br>hacer el resumen con los 3 enunciados. | Es un modelo que puede funcionar en audios cortos y preferentemente en inglés. |
| openai/whisper-tiny | Bajo desempeño, cambia muchas palabras por palabras inexistentes, muchos de los enunciados no hacen sentido. | Genera de manera correcta el resumen, sin embargo, lo simplifica demasiado, obteniendo solo una idea muy general <br>de la historia. | Modelo sencillo y rápido que no requiere de mucha capacidad de <br>procesamiento. ütil si no se cuenta con una GPU. | Muchos errores en la transcripción a texto, generación <br>de palabras inexistentes. | No es recomedable usar este modelo si lo que se busca es una transcripción <br>precisa de audio a texto. |
| openai/ whisper-medium | Buen desempeño, pero no perfecto. Presenta dificultades para identificar algunas palabras, además de juntar dos <br>palabras en una sola. Identifica bien signos de puntoación y énfasis en algunas pababras, además de identificar <br>cuando se hace referencia a que algún personaje dice un diálogo. | Genera de manera correcta el resumen, mencionando la idea principal además de los personajes principales <br>involucrados en la historia. | El modelo es bueno realizando una transcripción fiel de audio a <br>texto, además de requerir recursos computacionales moderados y <br>ser ágil con los recursos correctos.| Requiere el uso de GPU para tener un procesamiento <br>ágil, en caso contrario, es demasiado tardado. En el <br>caso de que no se cuente con GPU, la transcripción no <br>es la mejor pese a los largos tiempos de procesamiento <br>que puede tomar. | Es un modelo sumamente recomendable si se cuenta con GPU, intermedio en <br>cuanto a desempeño y tamaño.
| openai/whisper-base | Este modelo ofrece un rendimiento mejorado en términos de precisión de transcripción. | Es capaz de manejar mejor la variabilidad en el habla y el ruido de fondo, <br> mostrando una buena capacidad para transcribir audios en condiciones diversas. Sin embargo, sigue siendo menos <br>potente que los modelos más grandes.| Mejor manejo de distintos acentos y ruidos de fondo.<br> Aumenta la precisión general en la transcripción. <br>Aún conserva un tiempo de procesamiento razonable. | Aunque mejor que los modelos más pequeños, puede no <br>ser suficiente para entornos de alta exigencia. <br>Requiere más recursos que los modelos Tiny y Small. | Whisper Base es una opción sólida para <br>aplicaciones que necesitan un equilibrio entre velocidad y precisión,<br> siendo adecuado para audios con cierto nivel de complejidad pero no <br> en esta aplicación.|
| openai/whisper-large-v3 | Una versión actualizada del modelo Large, optimizada para mejorar la precisión y la eficiencia. | Ofrece mejoras sobre el modelo Large estándar, siendo aún más preciso y eficiente en el procesamiento de audio. |Mejoras en la precisión y velocidad en comparación con el modelo Large original.<br>Capacidad para manejar contextos variados de manera efectiva.<br>Menor latencia en la transcripción.|Requiere recursos computacionales significativos.<br>Puede no ser accesible para dispositivos con capacidades <br>limitadas. | Whisper Large v3 es una excelente opción con un alto nivel de <br>precisión en transcripción y un rendimiento eficiente en el procesamiento pero<br> no el más efectivo en esta aplicación|
| openai/whisper-large-v3 | Proporciona un balance entre la velocidad y la precisión, siendo más rápido que el modelo Large v3. | Proporciona un balance entre la velocidad y la precisión. Es más rápido que el modelo Large v3, lo que lo hace adecuado <br>para aplicaciones en tiempo real donde la velocidad de transcripción es importante. Aunque puede que no alcance el <br>mismo nivel de precisión que el modelo Large v3, suele ser suficiente para la mayoría de las aplicaciones, <br>ofreciendo una buena calidad de transcripción en condiciones variadas. | Alta precisión con una velocidad de procesamiento mejorada.<br>Ideal para aplicaciones en tiempo real donde se requiere velocidad.<br>Suficiente precisión para la mayoría de las aplicaciones prácticas. |Aunque es más rápido, puede no alcanzar el mismo nivel de precisión que el modelo Large v3.<br>Puede ser menos efectivo en contextos extremadamente complejos o ruidosos.|Whisper Large v3 Turbo es ideal para aplicaciones prácticas donde la velocidad es crucial,<br> proporcionando un buen compromiso entre precisión y rendimiento como es el caso de esta aplicación. |


##**Conclusión General**
La actividad de transcripción de audio a texto centrada en las fábulas ha demostrado que el modelo Whisper Large v3 Turbo es la opción más adecuada para esta aplicación, ya que ofrece un equilibrio óptimo entre precisión y velocidad. Su capacidad para manejar diferentes acentos y variaciones en el habla lo hace versátil y efectivo en la transcripción de textos narrativos, permitiendo resultados rápidos y precisos que son esenciales para el análisis de contenido. Además, su eficiencia en el uso de recursos computacionales lo hace accesible para diversas plataformas, convirtiéndolo en una herramienta ideal para proyectos que requieren transcripciones de alta calidad y un rendimiento confiable en el procesamiento de lenguaje natural.

# **Fin de la actividad LDA y LMM: audio-a-texto**